# DATA WRANGLING

## Step 1:  Data Importation and Initial Exploration

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin
import plotly.express as px
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
label = LabelEncoder()
scaler = StandardScaler()
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')

In [2]:
# Loading the Dataset and rounding numbers to 2 decimal places
school = pd.read_csv('data.csv', delimiter=';')
school.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
school['Target'].unique()

In [ ]:
# Checking the number of rows and columns
school.shape

In [ ]:
# Data types checking
school.dtypes

In [6]:
school.rename(columns={'Nacionality' :'Nationality', 'Daytime/evening attendance\t':'Daytime/evening attendance'},inplace=True)

In [ ]:
# Summary statistics
school.describe().T

## Step 2: Data Cleaning and Validation


In [ ]:
school['Target'].unique()

In [ ]:
sns.countplot(data=school,x='Target',hue='Target')
plt.show()

In [10]:
school = school[~school['Target'].isin(['Enrolled'])]

In [11]:
school['Target'] = np.where(school['Target']=='Dropout',1,0)

In [ ]:
# Checking for missing values
# school.isna().sum()
school.isna().sum()

In [13]:
numeric_1 = [
'Age at enrollment',
'Previous qualification (grade)',
'Admission grade',
'Curricular units 1st sem (credited)',
'Curricular units 1st sem (enrolled)',
'Curricular units 1st sem (evaluations)',
'Curricular units 1st sem (approved)',
'Curricular units 1st sem (grade)',
'Curricular units 1st sem (without evaluations)',
'Curricular units 2nd sem (credited)',
'Curricular units 2nd sem (enrolled)',
'Curricular units 2nd sem (evaluations)',
'Curricular units 2nd sem (approved)',
'Curricular units 2nd sem (grade)',
'Curricular units 2nd sem (without evaluations)',
'Unemployment rate',
'Inflation rate',
'GDP']


In [ ]:
school.shape

In [ ]:
school[numeric_1] = school[numeric_1].replace(0,np.nan)
school[numeric_1].isna().sum()

In [16]:
school.drop(['Curricular units 1st sem (credited)','Curricular units 2nd sem (credited)', 'Curricular units 1st sem (without evaluations)','Curricular units 2nd sem (without evaluations)'], axis=1, inplace=True)

In [ ]:
school.isna().sum()

In [18]:
numeric = [
'Age at enrollment',
'Previous qualification (grade)',
'Admission grade',
'Curricular units 1st sem (enrolled)',
'Curricular units 1st sem (evaluations)',
'Curricular units 1st sem (approved)',
'Curricular units 1st sem (grade)',
'Curricular units 2nd sem (enrolled)',
'Curricular units 2nd sem (evaluations)',
'Curricular units 2nd sem (approved)',
'Curricular units 2nd sem (grade)',
'Unemployment rate',
'Inflation rate',
'GDP']


categorical = ["Mother's occupation","Father's occupation", 'Marital status',
'Application mode','Course',"Previous qualification",'Application order',"Mother's qualification","Father's qualification",'Nationality']
oe = ["Mother's occupation","Father's occupation",
'Application mode','Course',"Mother's qualification","Father's qualification"]
le = ['Application order',"Previous qualification",'Marital status','Nationality']

In [19]:
for c in categorical:
  school[c] = school[c].astype(str)

In [ ]:
data =school[school['Curricular units 1st sem (enrolled)'].isna()]
data

In [ ]:
no_details = school[(school['Curricular units 1st sem (approved)'].isna()) & (school['Curricular units 1st sem (enrolled)'].isna()) & (school['Curricular units 1st sem (grade)'].isna())]
no_details
# notnull =school[school['Curricular units 1st sem (credited)'].notnull()][['Course','Curricular units 1st sem (credited)','Curricular units 1st sem (approved)','Curricular units 1st sem (grade)','Curricular units 1st sem (without evaluations)']]

In [ ]:
no_details['Course'].unique()

In [ ]:
school_df = school[school['Course']!='171']
school_df.isna().sum()

In [ ]:
school_df[(school_df['Curricular units 1st sem (approved)'].notna()) & school_df['Curricular units 1st sem (evaluations)'].isna()]

In [ ]:
school_df[(school_df['Curricular units 1st sem (grade)'].notna()) & school_df['Curricular units 1st sem (evaluations)'].isna()]

In [ ]:
school_df[(school_df['Curricular units 1st sem (grade)'].isna()) & school_df['Curricular units 1st sem (approved)'].notna()]

In [ ]:
school_df[numeric] = school_df[numeric].replace(0,np.nan)
school_df[numeric].isna().sum()

In [ ]:
ms = school_df['Marital status'].value_counts(normalize=True)
ms

In [ ]:
school_df.loc[school_df['Marital status'].isin(ms[ms<0.1].index),'Marital status'] = 'others'
school_df['Marital status'].value_counts(normalize=True)

In [ ]:
pq = school_df["Previous qualification"].value_counts(normalize=True)
pq

In [ ]:

school_df.loc[school_df["Previous qualification"].isin(pq[pq<0.05].index),"Previous qualification"] = 'others'
school_df["Previous qualification"].value_counts(normalize=True)

In [32]:
crs = school["Course"].value_counts(normalize=True)

In [ ]:
mo = school_df["Mother's occupation"].value_counts(normalize=True)
mo

In [ ]:

school_df.loc[school_df["Mother's occupation"].isin(mo[mo<0.01].index),"Mother's occupation"] = 'others'
school_df["Mother's occupation"].value_counts(normalize=True)

In [ ]:
fo = school_df["Father's occupation"].value_counts(normalize=True)
fo

In [ ]:
school_df.loc[school_df["Father's occupation"].isin(fo[fo<0.01].index),"Father's occupation"] = 'others'
school_df["Father's occupation"].value_counts(normalize=True)

In [ ]:
mq = school_df["Mother's qualification"].value_counts(normalize=True)
mq

In [ ]:
school_df.loc[school_df["Mother's qualification"].isin(mq[mq<0.01].index),"Mother's qualification"] = 'others'
school_df["Mother's qualification"].value_counts(normalize=True)

In [ ]:
fq = school_df["Father's qualification"].value_counts(normalize=True)
fq

In [ ]:
school_df.loc[school_df["Father's qualification"].isin(fq[fq<0.01].index),"Father's qualification"] = 'others'
school_df["Father's qualification"].value_counts(normalize=True)

In [ ]:
nat = school_df["Nationality"].value_counts(normalize=True)
nat

In [ ]:
school_df.loc[school["Nationality"].isin(nat[nat<0.9].index),"Nationality"] = 'others'
school_df["Nationality"].value_counts(normalize=True)


In [ ]:
am = school_df["Application mode"].value_counts(normalize=True)
am

In [ ]:
school_df.loc[school["Application mode"].isin(am[am<0.01].index),"Application mode"] = 'others'
school_df["Application mode"].value_counts(normalize=True)

In [ ]:
school_df['Target'].value_counts()

In [ ]:
school_df[numeric] = school_df[numeric].replace(np.nan,0)
school_df[numeric].isna().sum()

In [ ]:
school_df.info()

In [ ]:
school_df[numeric].hist(figsize=(30,20),layout=(7,6))
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
school_df[numeric].boxplot()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Checking for duplicates
school_df.duplicated().sum()

In [ ]:
school_df.info()

## Step 3: Data Transformation

In [52]:
# scale = scaler.fit_transform(school_df[numeric])
# school_df[numeric] = scale
# school_df['Admission grade'] = np.log(school_df['Admission grade'])

In [53]:
import pickle

In [ ]:
oh = onehot.fit_transform(school_df[oe])
school_df = pd.concat([school_df,oh],axis=1)
for c in le:
    school_df[c] = label.fit_transform(school_df[c])
school_df.drop(columns=oe,inplace=True)
school_df.info()

encoding ={'label':label, 'onehot':onehot}
with open('saved_encode.pkl','wb') as file:
    pickle.dump(encoding,file)


## Step 4:  Statistical Analysis

In [ ]:
# Descriptive Analysis
school_df.describe()

In [ ]:
# Correlation Analysis
fig = plt.figure(figsize=(10,5))
sns.heatmap(school_df.corr())
plt.show()

Hypotheses
1. Higher socio-economic status correlates with lower dropout rates.
2. Students with higher admission grades are less likely to drop out.
3. Dropout rates are lower among students receiving financial aid or scholarships.

### 2. Students with higher admission grades are less likely to drop out.
Ho: Admission grades are the same for all groups.\
Ha: Students with higher admission grades are less likely to drop out.

In [ ]:
sns.histplot(data=school_df,x='Admission grade',hue='Target',multiple='stack')
plt.show()

In [58]:
school_df['a'] = pd.qcut(school_df['Admission grade'],6)
school_df.groupby('a')['Target'].mean()
school_df.drop('a',axis=1,inplace=True)

In [ ]:
school_df['AdmGrades_category'] = np.where(school_df['Admission grade']<140,0,1)
school_df['AdmGrades_category'].value_counts()

In [ ]:
school_df.groupby('Target')['AdmGrades_category'].value_counts()

In [61]:
n_highest = np.array([391,223])
n_rows = np.array([391+1722,223+1116])

In [ ]:
alpha = 0.05
z_score,p_value = proportions_ztest(count = n_highest,nobs = n_rows, alternative='smaller')
print(z_score)
print(p_value)


The p-value is higher than a significant level of 0.05 so we fail to reject the null hypothesis and conclude that Admission grades are the same for all groups.

In [ ]:
# Visualizing Target categories and Tuition fees up to date
sns.kdeplot(data=school_df, x='Admission grade',hue='Target', cut=0)
plt.show()

### 3. Dropout rates are lower among students receiving financial aid or scholarships.\

Ho: Students receiving financial aid or scholarships and Dropout rates are independent.\
Ha: Students receiving financial aid or scholarships and Dropout rates are not independent.

In [ ]:
# Target proportions
school_df['Target'].value_counts(normalize=True)

In [ ]:
# Visualizing Target categories and Tuition fees up to date
sns.barplot(data=school_df, y='Target',hue='Scholarship holder', ci=None)
plt.show()

In [ ]:
props = school_df.groupby('Scholarship holder')['Target'].value_counts(normalize=True)
wide_props = props.unstack()
wide_props

In [ ]:
# Visualizing stacked bar of Target categories and Scholarship holder
props = school_df.groupby('Scholarship holder')['Target'].value_counts(normalize=True)
wide_props = props.unstack()
wide_props.plot(kind='bar',stacked=True)

In [ ]:
alpha = 0.05
expected, observed, stats = pingouin.chi2_independence(data=school_df, x='Scholarship holder',y='Target')
print(stats[stats['test']=='pearson']['pval'])

The p-value is less than a significant level of 0.05 so we reject the null hypothesis and conclude that Students receiving financial aid or scholarships and Dropout rates are not independent.

## Step 5

In [69]:
school_df.to_csv('cleaned_data.csv')